<a id="introduction"></a>
# Ciencia de Datos Acelerada con GPU
## Introducción a cuDF para usuarios de pandas
#### Presentador: Rommel Quintanilla (BLAZINGSQL)

Notebook adaptado a español desde [02_Introduction_to_cuDF_for_pandas.ipynb](https://github.com/BlazingDB/Welcome_to_BlazingSQL_Notebooks/blob/master/webinars/rapids_intro/02_Introduction_to_cuDF_for_pandas.ipynb) por Justin Brandenburg. 

-------

En este Notebook presumiremos de los beneficios de usar cuDF. cuDF es una librería en Python de DataFrame GPU, para cargar, ejecutar joins, hacer agregaciones, filtrar y toda clase de manipulación de data tabular usando un DataFrame style API.

# Primero, sumemos las librerías necesarias

In [1]:
import os

import pandas as pd
import numpy as np

import cupy as cp
import cudf as cd

# Importemos data desde un archivo csv

### **movies_pdf** es nuestro DataFrame de Pandas

In [2]:
movies_pdf = pd.read_csv("s3://bsql/data/rapids_intro/movies.csv")

### **movies_cdf** es nuestro cuDF

In [3]:
movies_cdf = cd.read_csv("s3://bsql/data/rapids_intro/movies.csv", storage_options={'anon': True})

# Ejemplos de estadísticas en dataset

In [4]:
print(movies_pdf.shape)
print(movies_pdf.ndim)
print(len(movies_pdf))

(4916, 28)
2
4916


In [5]:
print(movies_cdf.shape)
print(movies_cdf.ndim)
print(len(movies_cdf))

(4916, 28)
2
4916


# Explorando la Data

In [6]:
movies_pdf.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [7]:
movies_cdf.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,<NA>,Doug Walker,<NA>,<NA>,131.0,<NA>,Rob Walker,131.0,<NA>,Documentary,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,12.0,7.1,<NA>,0


In [8]:
movies_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4897 non-null   object 
 1   director_name              4814 non-null   object 
 2   num_critic_for_reviews     4867 non-null   float64
 3   duration                   4901 non-null   float64
 4   director_facebook_likes    4814 non-null   float64
 5   actor_3_facebook_likes     4893 non-null   float64
 6   actor_2_name               4903 non-null   object 
 7   actor_1_facebook_likes     4909 non-null   float64
 8   gross                      4054 non-null   float64
 9   genres                     4916 non-null   object 
 10  actor_1_name               4909 non-null   object 
 11  movie_title                4916 non-null   object 
 12  num_voted_users            4916 non-null   int64  
 13  cast_total_facebook_likes  4916 non-null   int64

In [9]:
movies_cdf.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   color                      4897 non-null   object
 1   director_name              4814 non-null   object
 2   num_critic_for_reviews     4867 non-null   float64
 3   duration                   4901 non-null   float64
 4   director_facebook_likes    4814 non-null   float64
 5   actor_3_facebook_likes     4893 non-null   float64
 6   actor_2_name               4903 non-null   object
 7   actor_1_facebook_likes     4909 non-null   float64
 8   gross                      4054 non-null   float64
 9   genres                     4916 non-null   object
 10  actor_1_name               4909 non-null   object
 11  movie_title                4916 non-null   object
 12  num_voted_users            4916 non-null   int64
 13  cast_total_facebook_likes  4916 non-null   int64
 14  acto

In [10]:
movies_pdf.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [11]:
movies_cdf.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

# Seleccionando subconjuntos del DataFrame

### Seleccionando campos que son data continua

In [12]:
movies_pdf.select_dtypes(include="number").head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204,4834,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78,33000
1,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220,48350,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35,0
2,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868,11700,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35,85000
3,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,NaN,131.0,NaN,131.0,NaN,8,143,0.0,NaN,NaN,NaN,12.0,7.1,NaN,0


In [13]:
movies_cdf.select_dtypes(include="number").head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204,4834,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78,33000
1,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220,48350,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35,0
2,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868,11700,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35,85000
3,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,<NA>,<NA>,131.0,<NA>,131.0,<NA>,8,143,0.0,<NA>,<NA>,<NA>,12.0,7.1,<NA>,0


### Seleccionando campos que son floats

In [14]:
movies_pdf.select_dtypes(include="float").head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio
0,723.0,178.0,0.0,855.0,1000.0,760505847.0,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78
1,302.0,169.0,563.0,1000.0,40000.0,309404152.0,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35
2,602.0,148.0,0.0,161.0,11000.0,200074175.0,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35
3,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35
4,NaN,NaN,131.0,NaN,131.0,NaN,0.0,NaN,NaN,NaN,12.0,7.1,NaN


In [15]:
movies_cdf.select_dtypes(include="float").head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio
0,723.0,178.0,0.0,855.0,1000.0,760505847.0,0.0,3054.0,237000000.0,2009.0,936.0,7.9,1.78
1,302.0,169.0,563.0,1000.0,40000.0,309404152.0,0.0,1238.0,300000000.0,2007.0,5000.0,7.1,2.35
2,602.0,148.0,0.0,161.0,11000.0,200074175.0,1.0,994.0,245000000.0,2015.0,393.0,6.8,2.35
3,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,0.0,2701.0,250000000.0,2012.0,23000.0,8.5,2.35
4,<NA>,<NA>,131.0,<NA>,131.0,<NA>,0.0,<NA>,<NA>,<NA>,12.0,7.1,<NA>


### Seleccionando campos que son valores discretos

In [16]:
movies_pdf.select_dtypes(include="object").head()

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating
0,Color,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,English,USA,PG-13
1,Color,Gore Verbinski,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,English,USA,PG-13
2,Color,Sam Mendes,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,English,UK,PG-13
3,Color,Christopher Nolan,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,English,USA,PG-13
4,NaN,Doug Walker,Rob Walker,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,NaN,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN


In [17]:
movies_cdf.select_dtypes(include="object").head()

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating
0,Color,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,English,USA,PG-13
1,Color,Gore Verbinski,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,English,USA,PG-13
2,Color,Sam Mendes,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,English,UK,PG-13
3,Color,Christopher Nolan,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,English,USA,PG-13
4,<NA>,Doug Walker,Rob Walker,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,<NA>,<NA>,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,<NA>,<NA>,<NA>


# Análisis de Data

### Resumen estadístico para todos los campos de data continua

In [18]:
movies_pdf.select_dtypes(include="number").describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4867.000000,4901.000000,4814.000000,4893.000000,4909.000000,4.054000e+03,4.916000e+03,4916.000000,4903.000000,4895.000000,4.432000e+03,4810.000000,4903.000000,4916.000000,4590.000000,4916.000000
mean,137.988905,107.090798,691.014541,631.276313,6494.488491,4.764451e+07,8.264492e+04,9579.815907,1.377320,267.668846,3.654749e+07,2002.447609,1621.923516,6.437429,2.222349,7348.294142
std,120.239379,25.286015,2832.954125,1625.874802,15106.986884,6.737255e+07,1.383222e+05,18164.316990,2.023826,372.934839,1.002427e+08,12.453977,4011.299523,1.127802,1.402940,19206.016458
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,49.000000,93.000000,7.000000,132.000000,607.000000,5.019656e+06,8.361750e+03,1394.750000,0.000000,64.000000,6.000000e+06,1999.000000,277.000000,5.800000,1.850000,0.000000
50%,108.000000,103.000000,48.000000,366.000000,982.000000,2.504396e+07,3.313250e+04,3049.000000,1.000000,153.000000,1.985000e+07,2005.000000,593.000000,6.600000,2.350000,159.000000
75%,191.000000,118.000000,189.750000,633.000000,11000.000000,6.110841e+07,9.377275e+04,13616.750000,2.000000,320.500000,4.300000e+07,2011.000000,912.000000,7.200000,2.350000,2000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,4.200000e+09,2016.000000,137000.000000,9.500000,16.000000,349000.000000


In [19]:
movies_cdf.select_dtypes(include="number").describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,4867.000000,4901.000000,4814.000000,4893.000000,4909.000000,4.054000e+03,4.916000e+03,4916.000000,4903.000000,4895.000000,4.432000e+03,4810.000000,4903.000000,4916.000000,4590.000000,4916.000000
mean,137.988905,107.090798,691.014541,631.276313,6494.488491,4.764451e+07,8.264492e+04,9579.815907,1.377320,267.668846,3.654749e+07,2002.447609,1621.923516,6.437429,2.222349,7348.294142
std,120.239379,25.286015,2832.954125,1625.874802,15106.986884,6.737255e+07,1.383222e+05,18164.316990,2.023826,372.934839,1.002427e+08,12.453977,4011.299523,1.127802,1.402940,19206.016458
min,1.000000,7.000000,0.000000,0.000000,0.000000,1.620000e+02,5.000000e+00,0.000000,0.000000,1.000000,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,49.000000,93.000000,7.000000,132.000000,607.000000,5.019656e+06,8.361750e+03,1394.750000,0.000000,64.000000,6.000000e+06,1999.000000,277.000000,5.800000,1.850000,0.000000
50%,108.000000,103.000000,48.000000,366.000000,982.000000,2.504396e+07,3.313250e+04,3049.000000,1.000000,153.000000,1.985000e+07,2005.000000,593.000000,6.600000,2.350000,159.000000
75%,191.000000,118.000000,189.750000,633.000000,11000.000000,6.110841e+07,9.377275e+04,13616.750000,2.000000,320.500000,4.300000e+07,2011.000000,912.000000,7.200000,2.350000,2000.000000
max,813.000000,511.000000,23000.000000,23000.000000,640000.000000,7.605058e+08,1.689764e+06,656730.000000,43.000000,5060.000000,4.200000e+09,2016.000000,137000.000000,9.500000,16.000000,349000.000000


### Resumen estadístico para todos los campos con valores discretos

In [20]:
movies_pdf.select_dtypes(include="object").describe()

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating
count,4897,4814,4903,4916,4909,4916,4893,4764,4916,4904,4911,4616
unique,2,2397,3030,914,2095,4916,3519,4756,4916,47,65,18
top,Color,Steven Spielberg,Morgan Freeman,Drama,Robert De Niro,The Mist,Steve Coogan,based on novel,http://www.imdb.com/title/tt0265632/?ref_=fn_t...,English,USA,R
freq,4693,26,18,233,48,1,8,4,1,4582,3710,2067


In [21]:
movies_cdf.select_dtypes(include="object").describe()

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating
count,4897,4814,4903,4916,4909,4916,4893,4764,4916,4904,4911,4616
unique,3,2398,3031,914,2096,4916,3520,4757,4916,48,66,19
top,Color,Steven Spielberg,Morgan Freeman,Drama,Robert De Niro,#Horror,Steve Coogan,based on novel,http://www.imdb.com/title/tt0006864/?ref_=fn_t...,English,USA,R
freq,4693,26,18,233,48,1,8,4,1,4582,3710,2067


### Transpuesta de los resultados de la función describe() de cuDF

(esto se puede hacer también en Pandas)

In [22]:
movies_cdf.select_dtypes(include="number").describe().T

,count,mean,std,min,25%,50%,75%,max
num_critic_for_reviews,4867.0,1.379889e+02,1.202394e+02,1.00,49.00,108.00,191.00,8.130000e+02
duration,4901.0,1.070908e+02,2.528601e+01,7.00,93.00,103.00,118.00,5.110000e+02
director_facebook_likes,4814.0,6.910145e+02,2.832954e+03,0.00,7.00,48.00,189.75,2.300000e+04
actor_3_facebook_likes,4893.0,6.312763e+02,1.625875e+03,0.00,132.00,366.00,633.00,2.300000e+04
actor_1_facebook_likes,4909.0,6.494488e+03,1.510699e+04,0.00,607.00,982.00,11000.00,6.400000e+05
gross,4054.0,4.764451e+07,6.737255e+07,162.00,5019656.25,25043962.00,61108412.75,7.605058e+08
num_voted_users,4916.0,8.264492e+04,1.383222e+05,5.00,8361.75,33132.50,93772.75,1.689764e+06
cast_total_facebook_likes,4916.0,9.579816e+03,1.816432e+04,0.00,1394.75,3049.00,13616.75,6.567300e+05
facenumber_in_poster,4903.0,1.377320e+00,2.023826e+00,0.00,0.00,1.00,2.00,4.300000e+01
num_user_for_reviews,4895.0,2.676688e+02,3.729348e+02,1.00,64.00,153.00,320.50,5.060000e+03


### Cálculo de Covarianza de dos variables continuas

In [23]:
movies_pdf.movie_facebook_likes.cov(movies_pdf.actor_3_facebook_likes)

8481703.754193125

In [24]:
movies_cdf.movie_facebook_likes.cov(movies_cdf.actor_3_facebook_likes)

8481703.754193125

### Correlación Pearson de dos variables continuas

In [25]:
movies_pdf.movie_facebook_likes.corr(movies_pdf.actor_3_facebook_likes)

0.27107033354561866

In [26]:
movies_cdf.movie_facebook_likes.corr(movies_cdf.actor_3_facebook_likes)

0.27107033354561866

### Agrupaciones
Analizando los montos brutos (gross amounts) generados por los dos actores principales

In [27]:
movies_pdf[['actor_1_name','actor_2_name','gross']].groupby(['actor_1_name','actor_2_name']).sum()

,,gross
actor_1_name,actor_2_name,
50 Cent,Bill Duke,30981850.0
A.J. Buckley,Jack McGraw,123070338.0
Aaliyah,Lena Olin,30307804.0
Aasif Mandvi,Dequina Moore,13214030.0
Abbie Cornish,Gabourey Sidibe,14989761.0
...,...,...
Zoë Kravitz,Lily Rabe,43097652.0
Zuhair Haddad,Harry Lennix,0.0
Álex Angulo,Germán Alcarazu,0.0


In [28]:
movies_cdf[['actor_1_name','actor_2_name','gross']].groupby(['actor_1_name','actor_2_name']).sum()

,,gross
actor_1_name,actor_2_name,
50 Cent,Bill Duke,30981850.0
A.J. Buckley,Jack McGraw,123070338.0
Aaliyah,Lena Olin,30307804.0
Aasif Mandvi,Dequina Moore,13214030.0
Abbie Cornish,Gabourey Sidibe,14989761.0
...,...,...
Zoë Kravitz,Lily Rabe,43097652.0
Zuhair Haddad,Harry Lennix,<NA>
Álex Angulo,Germán Alcarazu,<NA>


# Preparación de la Data

Algunos géneros (genres) son resultado de la combinación de otros varios géneros. Por ejemplo: `Action|Adventure|Comedy|Fantasy|Sci-Fi`

In [29]:
print('There are ' + str(len(movies_pdf.genres.unique())) + ' genre combinations in the genres field')
print('Examples:\n', movies_pdf.genres.unique()[:10])

There are 914 genre combinations in the genres field
Examples:
 ['Action|Adventure|Fantasy|Sci-Fi' 'Action|Adventure|Fantasy'
 'Action|Adventure|Thriller' 'Action|Thriller' 'Documentary'
 'Action|Adventure|Sci-Fi' 'Action|Adventure|Romance'
 'Adventure|Animation|Comedy|Family|Fantasy|Musical|Romance'
 'Adventure|Family|Fantasy|Mystery' 'Action|Adventure']


## Dividiendo la columna Género (Genre) usando pandas

In [30]:
genres_pdf = movies_pdf.join(movies_pdf.genres.str.split('|', expand=True).add_prefix('genre_'))
genres_pdf

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,aspect_ratio,movie_facebook_likes,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,1.78,33000,Action,Adventure,Fantasy,Sci-Fi,None,None,None,None
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,2.35,0,Action,Adventure,Fantasy,None,None,None,None,None
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,2.35,85000,Action,Adventure,Thriller,None,None,None,None,None
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2.35,164000,Action,Thriller,None,None,None,None,None,None
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,0,Documentary,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,NaN,84,Comedy,Drama,None,None,None,None,None,None
4912,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,16.00,32000,Crime,Drama,Mystery,Thriller,None,None,None,None
4913,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,NaN,16,Drama,Horror,Thriller,None,None,None,None,None
4914,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,2.35,660,Comedy,Drama,Romance,None,None,None,None,None


## Dividiendo la columna Género (Genre) usando cuDF
cuDF no tiene la opción add_prefix() cuando dividimos una columna. Acá mostramos una opción para renombrar la columna en cuDF.

In [31]:
genre_fields = len(movies_cdf.genres.str.split('|', expand=True).columns)
print('There will be ' + str(genre_fields) + ' new columns that will be added into our dataframe\n')
genres_cdf = movies_cdf.join(movies_cdf.genres.str.split('|', expand=True))
genres_cdf.head()

There will be 8 new columns that will be added into our dataframe



,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,aspect_ratio,movie_facebook_likes,0,1,2,3,4,5,6,7
1728,Black and White,Lawrence Kasdan,65.0,112.0,759.0,933.0,Alfre Woodard,11000.0,4554569.0,Comedy|Drama,...,2.35,209,Comedy,Drama,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1729,Color,Don Mancini,148.0,88.0,64.0,363.0,Billy Boyd,1000.0,17016190.0,Comedy|Fantasy|Horror|Thriller,...,1.85,0,Comedy,Fantasy,Horror,Thriller,<NA>,<NA>,<NA>,<NA>
1730,Color,John Maybury,161.0,94.0,11.0,566.0,Jason Lewis,1000.0,6301131.0,Drama|Mystery|Sci-Fi|Thriller,...,2.35,0,Drama,Mystery,Sci-Fi,Thriller,<NA>,<NA>,<NA>,<NA>
1731,Color,Ron Clements,124.0,90.0,63.0,631.0,Frank Welker,49000.0,217350219.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,1.33,0,Adventure,Animation,Comedy,Family,Fantasy,Musical,Romance,<NA>
1732,Color,F. Gary Gray,349.0,167.0,473.0,303.0,Neil Brown Jr.,559.0,161029270.0,Biography|Crime|Drama|History|Music,...,2.35,76000,Biography,Crime,Drama,History,Music,<NA>,<NA>,<NA>


**Los nuevos nombres de columna son asignados a números, que no son strings**

In [32]:
col_numbers = genres_cdf.columns[-genre_fields:].to_list()
print(col_numbers)

[0, 1, 2, 3, 4, 5, 6, 7]


**Necesitas convertirlos a strings, asignarles un prefijo y convertir ambas listas a un diccionario para renombrar las nuevas columnas en cuDF**

In [33]:
new_col_names = ['genre_' + str(x) for x in col_numbers] 
print(new_col_names)

new_col_dict = dict(zip(col_numbers, new_col_names))
print(new_col_dict)

['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7']
{0: 'genre_0', 1: 'genre_1', 2: 'genre_2', 3: 'genre_3', 4: 'genre_4', 5: 'genre_5', 6: 'genre_6', 7: 'genre_7'}


In [34]:
genres_cdf = genres_cdf.rename(columns=new_col_dict)

In [35]:
genres_cdf.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,aspect_ratio,movie_facebook_likes,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7
1728,Black and White,Lawrence Kasdan,65.0,112.0,759.0,933.0,Alfre Woodard,11000.0,4554569.0,Comedy|Drama,...,2.35,209,Comedy,Drama,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1729,Color,Don Mancini,148.0,88.0,64.0,363.0,Billy Boyd,1000.0,17016190.0,Comedy|Fantasy|Horror|Thriller,...,1.85,0,Comedy,Fantasy,Horror,Thriller,<NA>,<NA>,<NA>,<NA>
1730,Color,John Maybury,161.0,94.0,11.0,566.0,Jason Lewis,1000.0,6301131.0,Drama|Mystery|Sci-Fi|Thriller,...,2.35,0,Drama,Mystery,Sci-Fi,Thriller,<NA>,<NA>,<NA>,<NA>
1731,Color,Ron Clements,124.0,90.0,63.0,631.0,Frank Welker,49000.0,217350219.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,1.33,0,Adventure,Animation,Comedy,Family,Fantasy,Musical,Romance,<NA>
1732,Color,F. Gary Gray,349.0,167.0,473.0,303.0,Neil Brown Jr.,559.0,161029270.0,Biography|Crime|Drama|History|Music,...,2.35,76000,Biography,Crime,Drama,History,Music,<NA>,<NA>,<NA>


# One Hot Encoding

## OHE usando Pandas
Aplicado a la columna genre_0

In [36]:
pd_ohe = pd.get_dummies(genres_pdf.genre_0, prefix='genre_0')
df = pd.concat([genres_pdf, pd_ohe], axis=1)
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genre_0_Game-Show,genre_0_History,genre_0_Horror,genre_0_Music,genre_0_Musical,genre_0_Mystery,genre_0_Romance,genre_0_Sci-Fi,genre_0_Thriller,genre_0_Western
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,0,0,0,0,0,0,0,0,0,0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,0,0,0,0,0,0,0,0,0,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,0,0,0,0,0,0,0,0,0,0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,0,0,0,0,0,0,0,0,0,0


## OHE usando cuDF
Aplicado a la columna genre_0

In [37]:
cdf = cd.get_dummies(genres_cdf, prefix='genre_0', columns=['genre_0'])
cdf.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genre_0_Game-Show,genre_0_History,genre_0_Horror,genre_0_Music,genre_0_Musical,genre_0_Mystery,genre_0_Romance,genre_0_Sci-Fi,genre_0_Thriller,genre_0_Western
1728,Black and White,Lawrence Kasdan,65.0,112.0,759.0,933.0,Alfre Woodard,11000.0,4554569.0,Comedy|Drama,...,0,0,0,0,0,0,0,0,0,0
1729,Color,Don Mancini,148.0,88.0,64.0,363.0,Billy Boyd,1000.0,17016190.0,Comedy|Fantasy|Horror|Thriller,...,0,0,0,0,0,0,0,0,0,0
1730,Color,John Maybury,161.0,94.0,11.0,566.0,Jason Lewis,1000.0,6301131.0,Drama|Mystery|Sci-Fi|Thriller,...,0,0,0,0,0,0,0,0,0,0
1731,Color,Ron Clements,124.0,90.0,63.0,631.0,Frank Welker,49000.0,217350219.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,0,0,0,0,0,0,0,0,0,0
1732,Color,F. Gary Gray,349.0,167.0,473.0,303.0,Neil Brown Jr.,559.0,161029270.0,Biography|Crime|Drama|History|Music,...,0,0,0,0,0,0,0,0,0,0
